<a href="https://colab.research.google.com/github/mattiadido95/Haze-Fog-suppression/blob/main/dataset_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
from google.colab import drive
from google.colab.patches import cv2_imshow
import os
import gdown
import zipfile
import cv2
import numpy as np

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
folder_path = "drive/MyDrive/Haze-Fog-suppression"
os.chdir(folder_path)

# Uncompress Zip files


In [ ]:
os.chdir("dataset")
current_directory = os.getcwd()

#estrae tutti i file zip e cancella il file compresso
for file_name in os.listdir(current_directory):
    if file_name.endswith(".zip"):
        path_to_zip_file = os.path.join(current_directory, file_name)
        with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
            zip_ref.extractall(current_directory)
        os.remove(path_to_zip_file)

# Generate datasets

In [4]:
#`d` sia l'immagine di profondità e `thresholds` sia una lista di tre soglie
def compute_transmission_map_Koschmieder(d):
    H = d.shape[0]
    W = d.shape[1]
    vector = d.reshape(H * W * 3)
    for pixel_value in vector:
      if pixel_value < 50:
        pixel_value = 0
      elif pixel_value > 50 and pixel_value < 87:
        pixel_value = 50
      elif pixel_value > 87 and pixel_value < 167:
        pixel_value = 87
      elif pixel_value > 167 and pixel_value < 209:
        pixel_value = 167
      else:
        pixel_value = 255
    normalized = vector.reshape(H, W, 3).astype(np.uint8)
    t = np.exp(-0.01 * normalized)
    return t

def compute_transmission_map_Bruneton(d):
  beta = 0.008
  cos_theta = -0.8
  epsilon = 2
  t = np.exp(-beta * d / pow(cos_theta, epsilon))
  return t


In [43]:
def compute_cevr(original_image,degraded_image):
    max_J = np.max(original_image)
    min_J = np.min(original_image)
    max_I = np.max(degraded_image)
    min_I = np.min(degraded_image)

    cevr = (max_J - min_J) / (max_I - min_I)
    return cevr

In [5]:
# Specifica i parametri
# Valore di luce atmosferica (generalmente tra 0 e 255)
low_A = 120
medium_A = 180
high_A = 220

low_beta = 0.004
medium_beta = 0.006
high_beta = 0.008
very_high_beta = 0.01

path_to_original_images = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/Vis/"
path_to_depth_maps = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/DepthMaps/"
path_to_input = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/train_A/"
path_to_output = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/train_B/"

In [6]:
def generate_fog_image(path_to_original_image,path_to_depth_map,A,beta,img_name,level):
    # Carica l'immagine originale e la depth map
    J = cv2.imread(path_to_original_image)
    d = cv2.imread(path_to_depth_map)

    t = np.exp(-beta * d)
    I = J * t + A * (1 - t)

    cv2.imwrite(path_to_input + img_name[:-4] + "_" + str(level) + '.png', np.uint8(I))
    cv2.imwrite(path_to_output + img_name[:-4] + "_" + str(level) + '.png', np.uint8(J))

In [7]:
# Scorri tutti i file nella cartella
file_ordinati = sorted(os.listdir(path_to_original_images))

for nome_file in file_ordinati[:2100]:
    path_to_original_image = os.path.join(path_to_original_images, nome_file)
    path_to_depth_map = os.path.join(path_to_depth_maps, nome_file)
    generate_fog_image(path_to_original_image,path_to_depth_map,low_A,low_beta,nome_file,"low")
    generate_fog_image(path_to_original_image,path_to_depth_map,medium_A,medium_beta,nome_file,"medium")
    generate_fog_image(path_to_original_image,path_to_depth_map,high_A,high_beta,nome_file,"high")

In [ ]:
# Scorri tutti i file nella cartella
file_ordinati = sorted(os.listdir(path_to_original_images))

for nome_file in file_ordinati[2100:]:
    path_to_original_image = os.path.join(path_to_original_images, nome_file)
    path_to_depth_map = os.path.join(path_to_depth_maps, nome_file)
    generate_fog_image(path_to_original_image,path_to_depth_map,low_A,low_beta,nome_file,"low")
    generate_fog_image(path_to_original_image,path_to_depth_map,medium_A,medium_beta,nome_file,"medium")
    generate_fog_image(path_to_original_image,path_to_depth_map,high_A,high_beta,nome_file,"high")